# 10 Minute Tour of PyTestLab

Welcome to PyTestLab! This short tour will guide you through the basic setup and usage of PyTestLab to interact with your lab instruments.

## 1. Installation

PyTestLab can be installed using pip. If you haven't already, open your terminal and run:

In [1]:
!pip install pytestlab

Make sure you have the necessary VISA libraries installed for your operating system if you plan to connect to real instruments (e.g., NI-VISA, Keysight Connection Expert).

## 2. Importing PyTestLab and Connecting to an Instrument

PyTestLab uses an `AutoInstrument` class to easily load instrument configurations and connect. Configurations can be loaded from a CDN, local files, or a dictionary.

In [ ]:
from pytestlab.instruments import AutoInstrument
# from pytestlab.common.enums import DMMFunction # For DMM example
from pytestlab.experiments.database import MeasurementDatabase # For DB example
from pytestlab.experiments.results import MeasurementResult # For DB example

ImportError: cannot import name 'DMMFunction' from 'pytestlab.common.enums' (/home/remote/pytestlab/pytestlab/common/enums.py)

### Connecting to a Simulated Instrument

For this tour, we'll start with a simulated instrument, which doesn't require any physical hardware.

In [ ]:
def connect_simulated_scope():
    # Load a configuration for a Keysight DSOX1204G oscilloscope
    # 'simulate=True' tells AutoInstrument to use the simulation backend.
    scope_sim = AutoInstrument.from_config("keysight/DSOX1204G", simulate=True)
    
    # Connect to the backend (important step for instruments)
    scope_sim.connect_backend()
    
    print(f"Successfully connected to simulated: {scope_sim.id()}")
    return scope_sim

def main_sim():
    scope = connect_simulated_scope()
    # You can now interact with 'scope'
    scope.close() # Close the connection when done

# Simple function call - no asyncio needed
if __name__ == "__main__":
    main_sim()

### Connecting to a Real Instrument (Example)

To connect to a real instrument, you would set `simulate=False` and provide the VISA address. The `config_source` can be a path to a local YAML file or a CDN identifier.

In [ ]:
# This is an example and will likely fail if you don't have this instrument at this address.
def connect_real_psu():
    try:
        # Example: Connecting to a Keysight E36313A Power Supply
        psu_real = AutoInstrument.from_config(
            "keysight/E36313A",
            address="TCPIP0::192.168.1.100::INSTR",
            simulate=False
        )
        psu_real.connect_backend()
        print(f"Successfully connected to: {psu_real.id()}")
        return psu_real
    except Exception as e:
        print(f"Failed to connect to real PSU: {e}")
        return None

def main_real_example():
    psu = connect_real_psu()
    if psu:
        # Interact with the PSU
        psu.close()

# if __name__ == "__main__":
#     main_real_example()

## 3. Basic Operations using New Facades

PyTestLab now provides intuitive facade objects for common instrument operations, allowing for a chained, more readable syntax. These facade methods are synchronous and execute immediately.

### Power Supply (PSU) Facade Example

In [ ]:
def psu_facade_example():
    # Using a simulated PSU for this example
    psu = AutoInstrument.from_config("keysight/E36311A", simulate=True) # Example PSU config
    psu.connect_backend()
    print(f"PSU ID: {psu.id()}")

    try:
        # Using the PSUChannelFacade
        # Configure channel 1: set voltage to 3.3V, current limit to 0.5A, then turn it on.
        psu.channel(1).set(voltage=3.3, current_limit=0.5).on()
        print(f"PSU Channel 1: Voltage set to 3.3V, Current Limit to 0.5A, Output ON")

        # Read back the voltage (assuming the facade has get_voltage)
        # Note: The example PSUChannelFacade.set returns self, for get_voltage you'd call it separately.
        ch1_voltage = psu.channel(1).get_voltage()
        ch1_current = psu.channel(1).get_current()
        ch1_state = psu.channel(1).get_output_state()
        print(f"PSU Channel 1 Readback: Voltage={ch1_voltage}V, Current={ch1_current}A, State={'ON' if ch1_state else 'OFF'}")

        # Turn channel 1 off
        psu.channel(1).off()
        print(f"PSU Channel 1: Output OFF")
        ch1_state_after_off = psu.channel(1).get_output_state()
        print(f"PSU Channel 1 Readback after off: State={'ON' if ch1_state_after_off else 'OFF'}")

    except Exception as e:
        print(f"An error occurred with PSU facade: {e}")
    finally:
        psu.close()

# To run in Jupyter or script:
if __name__ == "__main__":
    psu_facade_example()

### Oscilloscope Facade Example

In [ ]:
from pytestlab.common.enums import TriggerSlope # Ensure TriggerSlope is imported

scope = AutoInstrument.from_config("keysight/DSOX1204G", simulate=True)
scope.connect_backend()
print(f"Scope ID: {scope.id()}")

try:
    # Channel Facade: Configure channel 1
    scope.channel(1).setup(scale=0.5, position=0.0, coupling='DC').enable()
    print("Scope Channel 1: Scale 0.5V/div, Position 0.0s, Coupling DC, Enabled")

    # Trigger Facade: Setup edge trigger on channel 1
    scope.trigger.setup_edge(source='CH1', level=1.0, slope=TriggerSlope.POSITIVE)
    print("Scope Trigger: Edge trigger on CH1, Level 1.0V, Slope POSITIVE")

    # Acquisition: Read waveform data
    print("Scope Acquisition: Starting...")
    waveform_data = scope.read_channels(1)
    print(f"Scope Acquisition: Waveform captured, data shape: {waveform_data.shape if hasattr(waveform_data, 'shape') else len(waveform_data)}")

except Exception as e:
    print(f"An error occurred with Oscilloscope facade: {e}")
finally:
    scope.close()

ValueError: Profile for device_type 'oscilloscope' is invalid: Profile validation failed: '1e-12' is not of type 'number'

### Waveform Generator (WG) Facade Example

In [ ]:
def wg_facade_example():
    wg = AutoInstrument.from_config("keysight/EDU33212A", simulate=True) # Example WG config
    wg.connect_backend()
    print(f"Waveform Generator ID: {wg.id()}")

    try:
        # Configure channel 1 to output a sine wave
        wg.channel(1).setup_sine(frequency=1e3, amplitude=1.0, offset=0.0).enable()
        print("WG Channel 1: Sine wave, 1kHz, 1.0Vpp, 0V offset, Output ON")

        # Example: Get some configuration (actual methods depend on WGChannelFacade implementation)
        # current_func = wg.channel(1).get_function_type() # Assuming such a method exists
        # current_freq = wg.channel(1).get_frequency()     # Assuming such a method exists
        # print(f"WG Channel 1 Readback: Function={current_func}, Frequency={current_freq}Hz")
        
        # Disable output
        wg.channel(1).disable()
        print("WG Channel 1: Output OFF")

    except Exception as e:
        print(f"An error occurred with Waveform Generator facade: {e}")
    finally:
        wg.close()

# To run in Jupyter or script:
if __name__ == "__main__":
    wg_facade_example()

## 4. Performing a Simple Measurement (DMM)

In [ ]:
def dmm_measurement_example():
    dmm = AutoInstrument.from_config("keysight/EDU34450A", simulate=True) # Example DMM config
    dmm.connect_backend()
    print(f"DMM ID: {dmm.id()}")

    measurement_result = None
    try:
        # Measure DC Voltage
        # The DMM 'measure' method takes a DMMFunction enum member.
        measurement_result = dmm.measure(DMMFunction.DC_VOLTAGE, range_val="AUTO", resolution="DEF")
        print(f"DMM Measurement: {measurement_result.measurement_type} = {measurement_result.values} {measurement_result.units}")
        
    except Exception as e:
        print(f"An error occurred during DMM measurement: {e}")
    finally:
        dmm.close()
    return measurement_result # Return for database example

# To run in Jupyter or script:
if __name__ == "__main__":
    dmm_result_for_db = dmm_measurement_example()

## 5. Storing and Retrieving Results with MeasurementDatabase

In [ ]:
def database_example(measurement_to_store):
    if not measurement_to_store:
        print("No measurement to store. Skipping database example.")
        # Create a dummy measurement if needed for the example to run
        measurement_to_store = MeasurementResult(
            values=1.234, 
            instrument="SimulatedDMM", 
            units="V", 
            measurement_type="DC Voltage (Dummy)"
        )
        print(f"Using dummy measurement: {measurement_to_store}")

    # Initialize the database (creates 'my_lab_data.db' if it doesn't exist)
    with MeasurementDatabase("my_lab_data") as db:
        print(f"Database initialized at: {db.db_path}")
        
        # Store the measurement result
        # store_measurement is synchronous
        try:
            measurement_codename = db.store_measurement(
                codename=None, # Let the DB generate a codename
                measurement=measurement_to_store,
                notes="This is a test measurement from the 10-minute tour."
            )
            print(f"Measurement stored with codename: {measurement_codename}")

            # Retrieve the stored measurement
            retrieved_measurement = db.retrieve_measurement(measurement_codename)
            print(f"Retrieved Measurement ({retrieved_measurement.instrument}): {retrieved_measurement.values} {retrieved_measurement.units}")
            print(f"Timestamp: {datetime.fromtimestamp(retrieved_measurement.timestamp)}")

            # List recent measurements
            recent_measurements = db.list_measurements(limit=5)
            print(f"\nRecent measurement codenames: {recent_measurements}")

            # Search for measurements
            search_results = db.search_measurements(query="tour")
            print(f"\nSearch results for 'tour':")
            for res in search_results:
                print(f"  - {res['codename']}: {res['measurement_type']} from {res['instrument']}")
        except Exception as e:
            print(f"Database operation failed: {e}")

# if __name__ == "__main__":
    # This assumes dmm_result_for_db was populated by the previous cell
    # In a real script, you'd pass results directly.
    # For this notebook, we'll call it directly if the variable exists.
    # if 'dmm_result_for_db' in locals() and dmm_result_for_db:
    #     database_example(dmm_result_for_db)    # else:
    #     print("DMM result not available for database example. Running with dummy data.")
    #     database_example(None) # Run with dummy data if DMM example didn't produce a result

# Simplified execution for notebook demonstration assuming previous cell ran
if __name__ == '__main__':
    # First, run the DMM example to get a result
    print("--- Running DMM Example First ---")
    dmm_res = asyncio.run(dmm_measurement_example())
    print("--- DMM Example Finished ---\n")
    
    print("--- Running Database Example ---")
    database_example(dmm_res)
    print("--- Database Example Finished ---")

## 6. Next Steps

This tour covered the basics of installing PyTestLab, connecting to instruments (simulated and real), using the new asynchronous facades for common operations, and storing/retrieving data.

From here, you can explore:
- Detailed documentation for specific instruments.
- Advanced features like experiment management and complex sweeps.
- Creating your own instrument configuration files.
- Integrating PyTestLab into your automated test scripts.